## Scrape the NASA Mars News Site

In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import os
import time

In [3]:
url = 'https://mars.nasa.gov/news'
chromedriver = "/usr/local/bin/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)
driver.get(url)
time.sleep(5)
html = driver.page_source

In [4]:
soup = BeautifulSoup(html, 'lxml')
results = soup.find_all('li', class_="slide")
text = results[0].find_all('a')
news_title = text[1].text
news_p = text[0].find('div', class_="rollover_description_inner").text

In [10]:
print(f"Title: {news_title}")
print("---------")
print(f"Paragraph: {news_p}")

Title: Mars Helicopter to Fly on NASA’s Next Red Planet Rover Mission
---------
Paragraph: NASA is adding a Mars helicopter to the agency’s next mission to the Red Planet, Mars 2020.


In [11]:
driver.close()

## Scrape JPL featured space image

In [12]:
from splinter import Browser

In [13]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [14]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [15]:
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(5)

In [16]:
browser.click_link_by_partial_text('more info')
links_found = browser.find_link_by_partial_href('images/largesize')
featured_image_url = links_found['href']

In [19]:
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14254_hires.jpg'

In [20]:
browser.quit()

## Scrape Mars weather

In [21]:
url = 'https://twitter.com/marswxreport?lang=en'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [25]:
results = soup.find_all('div', class_="js-tweet-text-container")

In [26]:
mars_weather = results[0].find('p').text

In [27]:
mars_weather

'Sol 2047 (May 10, 2018), Sunny, high 3C/37F, low -71C/-95F, pressure at 7.33 hPa, daylight 05:22-17:20'

## Scrape Mars facts using Pandas

In [28]:
import pandas as pd

In [29]:
url = 'https://space-facts.com/mars/'

In [30]:
tables = pd.read_html(url)

In [31]:
df = tables[0]

In [33]:
html_table = df.to_html(header=None,index=False)

In [34]:
html_table = html_table.replace('\n', '')

In [35]:
html_table

'<table border="1" class="dataframe">  <tbody>    <tr>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <td>Mass:</td>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <td>Surface Temperature:</td>      <td>-153 to 20 °C</td>    </tr>    <tr>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <td>Recorded By:</td>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

## Scrape USGS Astrogeology site and find Mars Hemisperes

In [36]:
hemisphere_image_urls =[]

In [37]:
def find_hemisperes(name):
    browser = Browser('chrome', **executable_path, headless=False)
    url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(url)
    browser.click_link_by_partial_text(name)
    links_found = browser.find_link_by_partial_href(name.split()[0].lower())
    url = links_found['href']
    dic = {"title": f"{name} Hemisphere", "img_url": url}
    hemisphere_image_urls.append(dic)
    browser.quit()

In [38]:
hemisperes_list = ['Cerberus', 'Schiaparelli', 'Syrtis Major', 'Valles Marineris']

In [39]:
for hemispere in hemisperes_list:
    find_hemisperes(hemispere)

In [40]:
hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere'}]